### Cell dedicated to imputation techniques

In [ ]:
import numpy as np
import pandas as pd
from typing import Tuple, Optional, List

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%run plot.ipynb

In [ ]:
class DistributionInfo:
  def __init__(self, vals):
    self.vals = vals
  
  def _mean(self):
    vals_lst = self.vals[~np.isnan(self.vals)]
    
    return sum(self.vals) / len(self.vals)
  
  def _moments(self, moment):
    
    vals_mean = self._mean()
    prod = [
      (val - vals_mean) ** moment 
      for val 
      in self.vals]
      
    return sum(prod) / len(self.vals)
  
  def _skew(self):
    
    len_vals = len(self.vals)
    moment_ratio = self._moments(3) / self._moments(2) ** 1.5

    return (len_vals * (len_vals - 1)) ** 0.5 / ((len_vals - 2)) * (moment_ratio)
  
  def _std_dev(self):
      mean = self._mean()
      citatel = [(val - mean) ** 2 for val in self.vals ]

      return (sum(citatel) / (len(self.vals) - 1)) ** 0.5
    
  def _kurtosis(self):
    
    return self._moments(4) / self._moments(2) ** 2
  
  def get_kurtosis_skewness(self):# -> Tuple[float, float]:
  
    return self._kurtosis(), self._skew()

In [ ]:
class ImpError:
  def __init__(self, actual: np.array, imputed: np.array):
    self.actual = actual
    self.imputed = imputed
    
   
  def _calculate_error_point(self, error_lst = []):
    for actual, imputed in list(zip(self.actual, self.imputed)):
      err = np.abs((imputed - actual)/actual)
      error_lst.append(err)

      #print(f"Actual: {actual} \t Imputed: {imputed} \t err: {err} \n")
      #np.savetxt(f"/dbfs/mnt/pbi/Bots/err_{name}.csv", error_lst, delimiter=",")
      
      
      
    return sum(error_lst) / len(self.imputed)
  

In [ ]:
class EM_impute(DistributionInfo):
    from typing import Optional, Tuple
    from scipy.stats import skewnorm
    
    def __init__(self, array, vals = None):
        self.array = array
        super().__init__(vals)

    def _get_from_normal(self):
        array = self.array[~np.isnan(self.array)]

        return np.random.normal(
            array.mean(), 
            array.std()
            )
        
    def _get_from_skewed_normal(self):
      
      kurtosis, skewness = self.get_kurtosis_skewness()
      std = self._std_dev()
      mean = self._mean()
      
      print(f"Sikmost rozdeleni: {skewness} \t a spicatost: {kurtosis} \n")
      
      return skewnorm.rvs(loc = mean, scale = std, size=1)
    
    def _get_from_uniform(
        self,
        min: float,
        max: float
    ):
    
      return np.random.uniform(min, max)
    
    def _impute_skewnorm(
                self,
                nan_array: np.array, 
                nan_index: np.array, 
                tol: Optional[float] = None
                ):
      
        val_lst = []
        index_lst = []
        prev  = 100000
        prvek = 0
        counter = 0

        for _ in nan_array:
            prvek += 1
            val = self._get_from_normal()
            delta = np.abs(val - prev) / prev
            val_lst.append(delta)
            index_lst.append(prvek)
            prev = val
          
            if tol:
                while np.abs(delta) > tol:
                  val = self._get_from_skewed_normal()
                  delta = np.abs(val - prev) / prev
                  prev = val
                  val_lst.append(delta)
                  index_lst.append(prvek)

                self.array[nan_index[prvek - 1]] = val
            else:
                while np.abs(delta) > 0.01:
                  val = self._get_from_skewed_normal()
                  delta =  np.abs(val - prev) / prev
                  prev = val
                  val_lst.append(delta)
                  index_lst.append(prvek)

                self.array[nan_index[prvek - 1]] = val

        return self.array, val_lst, index_lst

    def _impute_normal(
                self,
                nan_array: np.array, 
                nan_index: np.array, 
                tol: Optional[float] = None
                ):
      
        val_lst = []
        index_lst = []
        prev  = 100000
        prvek = 0
        counter = 0

        for _ in nan_array:
            prvek += 1
            val = self._get_from_normal()
            delta = np.abs(val - prev) / prev
            val_lst.append(delta)
            index_lst.append(prvek)
            prev = val
          
            if tol:
                while np.abs(delta) > tol:
                  val = self._get_from_normal()
                  delta = np.abs(val - prev) / prev
                  prev = val
                  val_lst.append(delta)
                  index_lst.append(prvek)

                self.array[nan_index[prvek - 1]] = val
            else:
                while np.abs(delta) > 0.01:
                  val = self._get_from_normal()
                  delta =  np.abs(val - prev) / prev
                  prev = val
                  val_lst.append(delta)
                  index_lst.append(prvek)

                self.array[nan_index[prvek - 1]] = val

        return self.array, val_lst, index_lst
      
    def _impute_uniform(
                self,
                nan_array: np.array, 
                nan_index: np.array, 
                tol: Optional[float] = None
                ):
        
        val_lst = []
        index_lst = []
        prev  = 100000
        prvek = 0

        for _ in nan_array:
            prvek += 1
            val = self._get_from_normal()
            delta = np.abs(val - prev) / prev
            val_lst.append(delta)
            index_lst.append(prvek)
            prev = val
          
            if tol:
                while np.abs(delta) > tol:
                  val = self._get_from_uniform()
                  delta = np.abs(val - prev) / prev
                  prev = val
                  val_lst.append(delta)
                  index_lst.append(prvek)               

                self.array[nan_index[prvek - 1]] = val
            else:
                while np.abs(delta) > 0.01:
                  val = self._get_from_uniform()
                  delta =  np.abs(val - prev) / prev
                  prev = val
                  val_lst.append(delta)
                  index_lst.append(prvek)

                self.array[nan_index[prvek - 1]] = val

        return self.array, val_lst, index_lst

    def em_imputation(self, **params) -> np.array:
      
      for k, v in params.items():
        print(f"{k} \t\t --> \t\t {v}")

      nan_array = self.array[np.isnan(self.array)]
      nan_index = np.argwhere(np.isnan(self.array))[:,0]

      if params["distribution"] == "normal":
          imputed_array, val_lst, index_lst = self._impute_normal(nan_array, nan_index)
      elif params["distribution"] == "dirichlet":
          imputed_array, val_lst, index_lst = self._impute_dirichlet(nan_array, nan_index, params["m"], params["n"], params["size"])
      elif params["distribution"] == "skew_normal":
          imputed_array, val_lst, index_lst = self._impute_normal(nan_array, nan_index)
      else:
        raise ValueError("Zadej vsechny hodnoty prosim.")

      return imputed_array, val_lst, index_lst

class Impute(EM_impute):
    def __init__(self, data, columns: List[str], ITER_TO_PLOT: int) :
        self.data = data
        self.columns = columns
        self.orig_data = self.data.copy()
        self.ITER_TO_PLOT = ITER_TO_PLOT

        super().__init__(None)
    
    def _create_missing_vals(self, col: List[str], num_vals):

        #d = self.data[col][0:self.num_vals].copy().to_numpy()
        self.data[col][0:num_vals] = np.nan

        self.array = self.data[col].to_numpy()

    
    def impute_cols(
                    self, 
                    num_vals,
                    EM_SKUTECNA = "Skutečná hodnota", 
                    EM_ODHADNUTA = "Hodnota odhanutá EM algoritmem", 
                    CHYBA_ODHADU = "Chyba odhadu chybějící hodnoty",
                    distribution: Optional[str] = None
                ):

        pplot = ImpPlot(
                SMALL_SIZE = 20, 
                BIG_SIZE = 16
            )

        

        for col in self.columns:
            self._create_missing_vals(col, num_vals)
            if distribution is not None:
              pred, err, index = self.em_imputation(distribution = distribution, tol = 0.001)
            else:
              pred, err, index = self.em_imputation(distribution = "normal", tol = 0.001)
            print(pred)
            
            error_normal = ImpError(
                                            actual = self.orig_data[col][0:num_vals],
                                            imputed = pred[0:num_vals]
                                    )
            print(error_normal._calculate_error_point())
            #print(pred)

            
            pplot.err_plot(
                            err, 
                            index, 
                            'o', 
                            CHYBA_ODHADU, 
                            iter_to_plot = self.ITER_TO_PLOT, 
                            y_lim = [-0.1, 0.15], 
                            units = '[K]', 
                            distribution = "normal", 
                            tol = '0.01', 
                            name = 'teplota_vzduchu'
                        )


            pplot.imp_plot(
                        y = self.orig_data[col][0:num_vals],
                        y2 = pred[0:num_vals],
                        y_color = 'o:r',
                        y_label = EM_SKUTECNA,
                        y2_color = 'o:b',
                        y2_label = EM_ODHADNUTA
                        )

        self.data[0:num_vals] = pred[0:num_vals][0]
        print(pred)

        return pred, self.data


In [ ]:
imp = Impute(
            data = pd.read_csv("./ai4i2020.csv"), 
            columns = ['Rotational speed [rpm]'], 
            ITER_TO_PLOT = 40
        )

        
pred, data = imp.impute_cols(distribution = "normal", num_vals = 1000)

df = pd.read_csv("ai4i2020.csv")
pred = pred[0:1000]
data = df['Rotational speed [rpm]'][0:1000]
np.average((data[1] - pred) / data)